<a href="https://colab.research.google.com/github/biggymart/colab/blob/main/02_TrainingAFeedForwardNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PyTorch Tutorial

https://www.youtube.com/watch?v=4p0G6tgNLis&t=28s

First ever Neuron Network!

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# 1 - download dataset
# 2 - create data loader
# 3 - build model
# 4 - train
# 5 - save trained model


BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

class FeedForwardNet(nn.Module):

  def __init__(self): # constructor
    super().__init__()
    self.flatten = nn.Flatten()
    self.dense_layers = nn.Sequential( # packing many layers into one block
        nn.Linear(28*28, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
    )
    self.softmax = nn.Softmax(dim=1)

  def forward(self, input_data): # data flow
    flattened_data = self.flatten(input_data)
    logits = self.dense_layers(flattened_data)
    predictions = self.softmax(logits)
    return predictions


def download_mnist_datasets():
  train_data = datasets.MNIST(
      root="data", # New folder called "data"
      download=True, # download plz if not already
      train=True, # it is a train dataset
      transform=ToTensor() # takes image in and reshapes it as tensor [0, 1]
  )
  validation_data = datasets.MNIST(
      root="data",
      download=True,
      train=False, # it is a non-train dataset
      transform=ToTensor()
  )
  return train_data, validation_data


def train_one_epoch(model, data_loader, loss_fn, optimizer, device):
  for inputs, targets in data_loader:
    inputs, targets = inputs.to(device), targets.to(device)

    # calculate loss
    predictions = model(inputs)
    loss = loss_fn(predictions, targets)

    # backpropagate loss and update weights
    optimizer.zero_grad() # reset gradients
    loss.backward() # backpropagation
    optimizer.step() # update weight

  print(f"[INFO] Loss: {loss.item()}")


def train(model, data_loader, loss_fn, optimizer, device, epochs):
  for i in range(epochs):
    print(f"[INFO] Epoch {i+1}")
    train_one_epoch(model, data_loader, loss_fn, optimizer, device)
    print("========================")
  print("[INFO] Training is done")


if __name__ == "__main__":
  # download MNIST dataset
  train_data, _ = download_mnist_datasets()
  print("[INFO] MNIST dataset downloaded")

  # create a data loader for the train set
  train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE)

  # build model
  if torch.cuda.is_available():
    device = "cuda"
  else:
    device = "cpu"
  print(f"[INFO] Using {device} device")
  feed_forward_net = FeedForwardNet().to(device) # [1] cuda [2] CPU

  # instantiate loss function + optimizer
  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(feed_forward_net.parameters(),
                               lr=LEARNING_RATE)

  # train model
  train(feed_forward_net, train_data_loader, loss_fn, optimizer, device, EPOCHS)

  # save model
  torch.save(feed_forward_net.state_dict(), "feedforwardnet.pth")
  print("[INFO] Model trained and stored at feedforwardnet.pth")

[INFO] MNIST dataset downloaded
[INFO] Using cpu device
[INFO] Epoch 1
[INFO] Loss: 1.5145989656448364
[INFO] Epoch 2
[INFO] Loss: 1.5001498460769653
[INFO] Epoch 3
[INFO] Loss: 1.4983819723129272
[INFO] Epoch 4
[INFO] Loss: 1.4825791120529175
[INFO] Epoch 5
[INFO] Loss: 1.476203441619873
[INFO] Epoch 6
[INFO] Loss: 1.473862648010254
[INFO] Epoch 7
[INFO] Loss: 1.4728718996047974
[INFO] Epoch 8
[INFO] Loss: 1.4731394052505493
[INFO] Epoch 9
[INFO] Loss: 1.4724836349487305
[INFO] Epoch 10
[INFO] Loss: 1.4737366437911987
[INFO] Training is done
[INFO] Model trained and stored at feedforwardnet.pth
